In [1]:
import tensorflow as tf
import numpy as np
import os
import time

2023-05-04 11:24:10.710524: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 11:24:11.106205: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-04 11:24:11.110356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 11:24:12.804543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Text length : {len(text)}')

Text length : 1115394


In [4]:
print(text[:150])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

A


In [5]:
vocab = sorted(set(text))
print(len(vocab))

65


In [6]:
examp_txt = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(examp_txt, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert = True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('UTF-8'))

F
i
r
s
t
 
C
i
t
i


2023-05-04 11:28:12.780077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


2023-05-04 11:29:25.171370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [19]:
for seq in sequences.take(10):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
b'zens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but th'
b'e superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we a'
b're too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particula'
b'rise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we bec'
b'ome rakes: for t

2023-05-04 11:30:13.090587: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [26]:
split_input_target(list("Testing"))

(['T', 'e', 's', 't', 'i', 'n'], ['e', 's', 't', 'i', 'n', 'g'])

In [27]:
dataset = sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target :", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target : b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2023-05-04 11:40:30.414082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


In [29]:
BATCH_SIZE = 32
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(32, 100), dtype=tf.int64, name=None), TensorSpec(shape=(32, 100), dtype=tf.int64, name=None))>

In [30]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 128
rnn_units = 512

In [32]:
class RNN_Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                      return_sequences=True,
                                      return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x

In [33]:
model = RNN_Model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

2023-05-04 12:10:46.364477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-05-04 12:10:46.365107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]


(32, 100, 66)


In [35]:
model.summary()

Model: "rnn__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  8448      
                                                                 
 gru (GRU)                   multiple                  986112    
                                                                 
 dense (Dense)               multiple                  33858     
                                                                 
Total params: 1,028,418
Trainable params: 1,028,418
Non-trainable params: 0
_________________________________________________________________


In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [37]:
sampled_indices

array([35, 22,  1, 56, 45, 60, 56, 49, 65,  9,  1, 46, 14, 54,  1, 21, 32,
       21, 34, 21, 30, 37, 17,  7, 29, 25,  4, 11, 47, 58, 34, 18, 29, 25,
       32, 27,  6, 56, 58, 45, 59, 11, 34, 12, 17, 41, 61, 24, 12, 51, 21,
       54, 28, 59, 48, 46, 55, 46, 40, 27, 51, 38, 20, 34,  9,  6, 63,  1,
       62, 61, 29, 58, 35, 17, 56, 20, 58, 25, 36, 14,  1, 34, 25, 16,  0,
       29, 52, 11, 25,  5, 27, 65, 44, 54, 36, 59, 34, 36, 12,  7])

In [38]:
print("Input: ", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:  b' would not buy\nTheir mercy at the price of one fair word;\nNor cheque my courage for what they can gi'

Next Char Predictions:
 b"VI\nqfuqjz.\ngAo\nHSHUHQXD,PL$:hsUEPLSN'qsft:U;DbvK;lHoOtigpgaNlYGU.'x\nwvPsVDqGsLWA\nULC[UNK]Pm:L&NzeoWtUW;,"


In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [41]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(example_batch_predictions.shape)
print(example_batch_mean_loss)

(32, 100, 66)
tf.Tensor(4.190918, shape=(), dtype=float32)


In [42]:
tf.exp(example_batch_mean_loss).numpy()

66.08343

In [43]:
model.compile(optimizer='adam', loss=loss)

In [45]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "checkpoint_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [46]:
EPOCHS = 5

In [47]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5


2023-05-04 12:25:40.407169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-05-04 12:25:40.407669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [1115394]
	 [[{{node Placeholder/_0}}]]
2023-05-04 12:25:40.682375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_d

345/345 [==============================] - 152s 435ms/step - loss: 2.4155
Epoch 2/5
345/345 [==============================] - 153s 442ms/step - loss: 1.8059
Epoch 3/5
345/345 [==============================] - 159s 459ms/step - loss: 1.5923
Epoch 4/5
345/345 [==============================] - 148s 430ms/step - loss: 1.4794
Epoch 5/5
345/345 [==============================] - 153s 443ms/step - loss: 1.4117


In [50]:
class Prediction_Model(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        
        predicted_logits = predicted_logits + self.prediction_mask
        
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        
        predicted_chars = self.chars_from_ids(predicted_ids)
        
        return predicted_chars, states

In [51]:
pred_model = Prediction_Model(model, chars_from_ids, ids_from_chars)

In [72]:
states = None
next_char = tf.constant(['Generated Text:'])
result = [next_char]

for n in range(1000):
    next_char, states = pred_model.generate_one_step(next_char, states=states)
    #print('my char ' + next_char)
    result.append(next_char)
    
result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)


Generated Text:
And so heardering oath thy masters, and you sir, how good?

BAPTISTA:
Alast you remember mine by the charantless
Tall mine excress, the rume conceive The coldon
A jest, away in mine own leaving bass.
Spoke's away of a clouds such a
sweet man, little Claudio, birth
How wound I lively my meant aponting gream our hand?

GRUMIO:
At, he smiles of fair Katharine,
Post dost like my kingly head, this is the lass! a man a lenttener,
stall's not conseating a fool, a
Tater, while and in rest.

LUCENTIO:
Horte good: you are gone.

Crown:
Nesty much: he may shail, now thank you again.

BAPTISTA:
I am his fellows tage; besides myself,--
Come, is he mady mistress! books and this thing need
Something to thy way with me.
Her corr? We will atter
That ip him so rocks were you; inhals
Werch me women a the fool:
For XETER:
'Busy it well, he'm'st
thou not, do altowen, sir, here, sir, he enter
Man life by gin time age.

BIONDELLO:
Masters Kathariant.

EUCELIET:
Or, that on his gentleman hath 